# FLOWCELL | OPERATION (facultatif)

[AAAA-MM-JJ] Suivi des opérations. 

Renseigner les noms de la _flowcell_ et de l'expérience (informations dans la SampleSheet `bs -c cac1 list runs --newer-than 3d --filter-term PRAG --filter-field ExperimentName`). Cette ligne et les suivantes de cette cellule sont à titre informatives et peuvent être supprimées du journal.

0. Mise-en-place de l'environnement de travail
    1. Déconvoluer et convertir les BCLs en FASTQs (automatique, `dragen_bcl-convert_watcher.sh`)
1. Préparer les cas `run_pipeline_prag.sh`
    1. Collecter les informations sur les familles
    2. Téléverser les FASTQs sur BaseSpace
    3. Générer le fichier "emg_batch_manifest.csv"
2. Glisser-déposer le fichier "emg_batch_manifest.csv" dans Emedgene
    1. S'authentifier sur [Emedgene](https://chusaintejustine.emedgene.com/)
    2. Cliquer sur _+ New Case > Switch to batch_ (coin supérieur droit)
    3. Glisser-déposer le fichier "emg_batch_manifest.csv" dans la section _Select file_
3. Collecter les metriques
4. Archiver les résultats
5. Nettoyer

_C.f._ `README-PRAGMatIQ_Process.md` pour plus de détails et le dépannage.

In [ ]:
## 0. Mise en place de l'environnement de travail
# ssh ${USER}@10.128.80.26
# Pour connaître l'identifiant FC et XP
bs -c cac1 list runs --newer-than 3d --filter-term PRAG --filter-field ExperimentName
screen -S prag

conda deactivate && conda activate CQGC-utils
export FC="" # Ex.: 20250613_LH00336_0223_A232CFLLT3
a=($(echo ${FC} | tr '_' '\n')) 
export BASEDIR="/mnt/vs_nas_chusj/CQGC_PROD/sequenceurs/${a[1]}"
export WORKDIR="/mnt/vs_nas_chusj/CQGC_PROD/fastqs"
export SOFTDIR='/mnt/spxp-app02/staging2/soft/CQGC-utils'


In [ ]:
## 1. Préparer les cas
## Le pipeline téléverse les FASTQs sur BaseSpace et prépare "emg_batch_manifest.csv",
## qu'il construit à partir des informations dans "samples_list.csv", prises de Nanuq.

if [[ ! -d "${WORKDIR}/${FC}" ]]; then mkdir ${WORKDIR}/${FC}; fi
cd ${WORKDIR}/${FC}
bash ${SOFTDIR}/Analysis.pipeline_illumina/run_pipeline_prag.sh ${FC} 2>&1 | tee ${WORKDIR}/${FC}/run_pipeline_prag.log

## 2. Glisser-déposer le fichier "emg_batch_manifest.csv" dans [Emedgene](https://chusaintejustine.emedgene.com/) (New Case > Switch to batch)

In [ ]:
## 3. Collecter les metriques

python /staging2/soft/CQGC-utils/Analysis.pipeline_illumina/emg_collect_samples_metrics.py ${FC}

In [ ]:
## 4. Archiver les résultats

ssh hien@narval.calculquebec.ca

samples=()
total_samples=${#samples[@]}

cd ${HOME}/projects/ctb-rallard/COMMUN/PRAGMatIQ-EMG
cp archive_PRAGMatIQ.log archive_PRAGMatIQ.log0

count=1
for sample in ${samples[@]}; do
    echo "Processing sample ${count}/${total_samples}, ${sample}"
    aws s3 --profile emedgene cp s3://cac1-prodca-emg-auto-results/CHU_Sainte_Justine/${sample}/ ./archives/${sample} --recursive
    echo ${sample} $( date +'%Y-%m-%d %T' ) >> archive_PRAGMatIQ.log
    ((count++))
done

In [ ]:
# 6. Nettoyer
# Avant d'effacer le répertoire de la run, otifier le laboratoire qui doit sauvegarder quelqeues métriques

`rm -R ${BASEDIR}/${FC}`

## Logs

[AAAA-MM-JJ] Renseigner ici les problèmes rencntrés lors du traitement de la run.

## Résolution des problèmes fréquents

Pour plus de détails, voir `README-PRAGMatIQ_Process.md`, s'il vous plaît.

### Récupérer les termes HPOs

`python ~/workspace/cqgc-dev/Helpers/get_phenotips_hpos.py P0000844`

### Déconvolution manuelle

Normalement, le demux est automatique avec "dragen_bcl-convert_watcher.sh" en cron. Si la déconvolution par cron n'a pas lieu, utiliser le code ci-dessous.

In [ ]:
## 0.1. Demux
## Normalement, le demux est automatique avec "dragen_bcl-convert_watcher.sh" en cron

# python ${SOFTDIR}/Helpers/get_nanuq_files.py --run ${FC}
# until [ -f "${BASEDIR}/${FC}/CopyComplete.txt" ]; do
#     printf '.'
#     sleep 900
# done
# qsub -V -o "${WORKDIR}/${FC}/qsub_out.txt" -e "${WORKDIR}/${FC}/qsub_err.txt" /staging2/soft/CQGC-utils/Helpers/dragen_bcl-convert_launcher.sh ${FC}
